<a href="https://colab.research.google.com/github/xmpuspus/Lectures/blob/master/notebooks/IntroTopicModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic Modeling  
As the name suggests, it is a process to automatically identify topics present in a text object and to derive hidden patterns exhibited by a text corpus. 

Topics can be defined as “a repeating pattern of co-occurring terms in a corpus”. A good topic model should result in – “health”, “doctor”, “patient”, “hospital” for a topic – Healthcare, and “farm”, “crops”, “wheat” for a topic – “Farming”.

Topic Models are very useful for the purpose for document clustering, organizing large blocks of textual data, information retrieval from unstructured text and feature selection. For Example – New York Times are using topic models to boost their user – article recommendation engines. Various professionals are using topic models for recruitment industries where they aim to extract latent features of job descriptions and map them to right candidates. They are being used to organize large datasets of emails, customer reviews, and user social media profiles.  

[Reference](https://www.analyticsvidhya.com/blog/2016/08/beginners-guide-to-topic-modeling-in-python/)

### Import Packages

In [0]:
import nltk
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

from gensim.models import CoherenceModel

# Plotting tools
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

nltk.download('stopwords')
nltk.download('wordnet')

    100% |████████████████████████████████| 1.6MB 14.0MB/s 
  Running setup.py bdist_wheel for pyLDAvis ... - \ done
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Preparing Documents  
We combine these documents to form a corpus.

In [0]:
doc1 = "Sugar is bad to consume. My sister likes to have sugar, but not my father."
doc2 = "My father spends a lot of time driving my sister around to dance practice."
doc3 = "Doctors suggest that driving may cause increased stress and blood pressure."
doc4 = "Sometimes I feel pressure to perform well at school, but my father never seems to drive my sister to do better."
doc5 = "Health experts say that Sugar is not good for your lifestyle."

# compile documents
doc_complete = [doc1, doc2, doc3, doc4, doc5]

### Cleaning and Preprocessing  
Remove the punctuations, stopwords and normalize the corpus.

In [0]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in doc_complete] 
doc_clean

[['sugar', 'bad', 'consume', 'sister', 'like', 'sugar', 'father'],
 ['father',
  'spends',
  'lot',
  'time',
  'driving',
  'sister',
  'around',
  'dance',
  'practice'],
 ['doctor',
  'suggest',
  'driving',
  'may',
  'cause',
  'increased',
  'stress',
  'blood',
  'pressure'],
 ['sometimes',
  'feel',
  'pressure',
  'perform',
  'well',
  'school',
  'father',
  'never',
  'seems',
  'drive',
  'sister',
  'better'],
 ['health', 'expert', 'say', 'sugar', 'good', 'lifestyle']]

### Preparing Document-Term Matrix

In [0]:
# Importing Gensim
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
doc_term_matrix

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2)],
 [(2, 1), (4, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)],
 [(8, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1)],
 [(2, 1),
  (4, 1),
  (18, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1)],
 [(5, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1)]]

### Running Latent Dirichlet Allocation (LDA) Model

In [0]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50, random_state=42)

### Results

Topic-Word Distribution

In [0]:
print(ldamodel.print_topics(num_topics=3, num_words=2))

[(0, '0.135*"sugar" + 0.054*"consume"'), (1, '0.079*"driving" + 0.045*"blood"'), (2, '0.057*"father" + 0.057*"sister"')]


Above results show that 3 topics. We can infer them through inspection:

Topic 0 -- Sugar  
Topic 1 -- Driving  
Topic 2 -- Family  

Document-Topic Distribution

In [0]:
topic_lists = ['Sugar', 'Driving', 'Family']

In [0]:
lists = ldamodel.get_document_topics(doc_term_matrix, minimum_probability=0.0)

In [0]:
print ('Document 1: ', doc1)
[(topic_lists[i[0]], i[1]) for i in lists[0]]

Document 1:  Sugar is bad to consume. My sister likes to have sugar, but not my father.


[('Sugar', 0.9128762), ('Driving', 0.043293275), ('Family', 0.04383049)]

In [0]:
print ('Document 2: ', doc2)
[(topic_lists[i[0]], i[1]) for i in lists[1]]

Document 2:  My father spends a lot of time driving my sister around to dance practice.


[('Sugar', 0.034941718), ('Driving', 0.93001276), ('Family', 0.0350455)]

In [0]:
print ('Document 3: ', doc3)
[(topic_lists[i[0]], i[1]) for i in lists[2]]

Document 3:  Doctors suggest that driving may cause increased stress and blood pressure.


[('Sugar', 0.033592276), ('Driving', 0.93213683), ('Family', 0.034270868)]

In [0]:
print ('Document 4: ', doc4)
[(topic_lists[i[0]], i[1]) for i in lists[3]]

Document 4:  Sometimes I feel pressure to perform well at school, but my father never seems to drive my sister to do better.


[('Sugar', 0.026379962), ('Driving', 0.026502648), ('Family', 0.9471174)]

In [0]:
print ('Document 5: ', doc5)
[(topic_lists[i[0]], i[1]) for i in lists[4]]

Document 5:  Health experts say that Sugar is not good for your lifestyle.


[('Sugar', 0.90422046), ('Driving', 0.047857955), ('Family', 0.04792159)]

### Compute Perplexity and Coherence Socre

In [0]:
# Compute Perplexity
print('\nPerplexity: ', ldamodel.log_perplexity(doc_term_matrix))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamodel, texts=doc_clean, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -4.158875516680784

Coherence Score:  0.35696487231896973


### Visualize Topics-Keywords

In [0]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      41.087006        1       1  0.078498 -0.046690
0      29.648777        1       2 -0.088689 -0.032254
2      29.264215        1       3  0.010191  0.078943, topic_info=     Category      Freq       Term     Total  loglift  logprob
term                                                          
5     Default  2.000000      sugar  2.000000  30.0000  30.0000
8     Default  1.000000    driving  1.000000  29.0000  29.0000
29    Default  1.000000       well  1.000000  28.0000  28.0000
27    Default  1.000000      seems  1.000000  27.0000  27.0000
22    Default  1.000000      drive  1.000000  26.0000  26.0000
26    Default  1.000000     school  1.000000  25.0000  25.0000
21    Default  1.000000     better  1.000000  24.0000  24.0000
23    Default  1.000000       feel  1.000000  23.0000  23.0000
24    Default  1.000000      never  1.000000  22.0000  22.0000
25    Default  1.000000    perform  1.000000  21.0000  21.0000
28    Default  1.000000  sometimes  1.000000  20.0000  20.0000
0     Default  1.000000        bad  1.000000  19.0000  19.0000
3     Default  1.000000       like  1.000000  18.0000  18.0000
1     Default  1.000000    consume  1.000000  17.0000  17.0000
30    Default  1.000000     expert  1.000000  16.0000  16.0000
31    Default  1.000000       good  1.000000  15.0000  15.0000
32    Default  1.000000     health  1.000000  14.0000  14.0000
33    Default  1.000000  lifestyle  1.000000  13.0000  13.0000
34    Default  1.000000        say  1.000000  12.0000  12.0000
18    Default  1.000000   pressure  1.000000  11.0000  11.0000
19    Default  1.000000     stress  1.000000  10.0000  10.0000
20    Default  1.000000    suggest  1.000000   9.0000   9.0000
16    Default  1.000000  increased  1.000000   8.0000   8.0000
15    Default  1.000000     doctor  1.000000   7.0000   7.0000
14    Default  1.000000      cause  1.000000   6.0000   6.0000
13    Default  1.000000      blood  1.000000   5.0000   5.0000
17    Default  1.000000        may  1.000000   4.0000   4.0000
12    Default  1.000000       time  1.000000   3.0000   3.0000
11    Default  1.000000     spends  1.000000   2.0000   2.0000
10    Default  1.000000   practice  1.000000   1.0000   1.0000
...       ...       ...        ...       ...      ...      ...
21     Topic3  0.707998     better  1.079159   0.8073  -2.8777
29     Topic3  0.707998       well  1.079159   0.8073  -2.8777
28     Topic3  0.707998  sometimes  1.079159   0.8073  -2.8777
27     Topic3  0.707998      seems  1.079159   0.8073  -2.8777
26     Topic3  0.707998     school  1.079159   0.8073  -2.8777
24     Topic3  0.707998      never  1.079159   0.8073  -2.8777
23     Topic3  0.707998       feel  1.079159   0.8073  -2.8777
22     Topic3  0.707998      drive  1.079159   0.8073  -2.8777
18     Topic3  0.709986   pressure  1.674811   0.3706  -2.8749
2      Topic3  0.712535     father  2.191751   0.1052  -2.8713
4      Topic3  0.712535     sister  2.191751   0.1052  -2.8713
33     Topic3  0.177258  lifestyle  1.065009  -0.5643  -4.2625
34     Topic3  0.177258        say  1.065009  -0.5643  -4.2625
30     Topic3  0.177258     expert  1.065009  -0.5643  -4.2625
31     Topic3  0.177258       good  1.065009  -0.5643  -4.2625
32     Topic3  0.177258     health  1.065009  -0.5643  -4.2625
1      Topic3  0.177256    consume  1.065008  -0.5643  -4.2626
3      Topic3  0.177256       like  1.065008  -0.5643  -4.2626
0      Topic3  0.177256        bad  1.065008  -0.5643  -4.2626
12     Topic3  0.177242       time  1.143877  -0.6359  -4.2626
11     Topic3  0.177242     spends  1.143877  -0.6359  -4.2626
10     Topic3  0.177242   practice  1.143877  -0.6359  -4.2626
9      Topic3  0.177242        lot  1.143877  -0.6359  -4.2626
7      Topic3  0.177242      dance  1.143877  -0.6359  -4.2626
6      Topic3  0.177242     around  1.143877  -0.6359  -4.2626
14     Topic3  0.177228      caus